In [1]:
# Define initial prompt and output format
initial_prompt = (
    "Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: // suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3. explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave blank for 'risk not present.'"
)

output_format_prompt = (
    "Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'"
)

In [2]:
# Define output schema
output_schema = {
    'key_to_extract': 'risk_output',
    'value_mapping': {
        'risk_present': 1,
        'risk_not_present': 0
    },
    'regex_pattern': r'"risk_output":\s*"(.*?)"'
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "risks.csv"

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['text', 'label'])
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=4, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (8, 2)
                                                text  label
0  Please confirm if the final balance on the acc...      0
1  Consumer said they are trying to avoid harming...      1
2  Good afternoon <person>, I refer to the compla...      0
3            Consumer had suicide attempt in January      1
4  Could you provide a gesture of goodwill for th...      0


In [8]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema,
    use_cache=True  # Set to False if you want to disable caching
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 17760
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention    │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: //           │
│ suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case  │
│ 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case         │
│ contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3.  │
│ explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your         │
│ explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave      │
│ blank for 'risk not present.'                                                                                   │
│                                                                                                                 │
│ Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and          │
│ explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/8
Using cached output for text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: This conversation just started. We have not discussed a case or any financial information previously. If you'd like to discuss an actual case, I'm happy to help.However, based on our previous conversation, it seems we were discussing identifying potential risks in a client's situation. If that's still relevant, please provide the text and I can attempt to assess whether the risk of self-harm is present.If not, feel free to start fresh, and let me know how I can assist you!
Processing text 2/8
Using cached output for text 2/8
Prediction 2/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/8
Using cached output for text 3/8
Unable to find risk_output in the raw output
Prediction 3/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: I can’t do that. Is there something else I can help you with?
Processi

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 1.0000 │
│ Recall              │ 0.7500 │
│ Accuracy            │ 0.7500 │
│ F1-score            │ 0.8571 │
│ Valid Predictions   │      4 │
│ Invalid Predictions │      4 │
└─────────────────────┴────────┘


Analyzing misclassifications, true positives, and invalid outputs...


╭───────── False Positives Analysis ──────────╮
│                                             │
│ No false positives found in this iteration. │
│                                             │
╰─────────────────────────────────────────────╯

╭─────────────────────────────────────────── False Negatives Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│ **Analysis of False Negative Examples**                                                                         │
│                                                                                                                 │
│ **Common Patterns or Characteristics:**                                                                         │
│                                                                                                                 │
│ 1. **Previous history of suicidal ideation**: The client has previously expressed suicidal thoughts, but there  │
│ is no current indication of risk.                                                                               │
│ 2. **Current absence of explicit suicidality language**: The text does not mention specific words or phrases    │
│ associated with suicidal ideation (e.g., "end it all," "not worth living").                                     │
│ 3. **Focus on existing medical conditions**: The client's primary concerns are related to their                 │
│ illness/disability, rather than suicidal thoughts.                                                              │
│                                                                                                                 │
│ **Subtle Indicators of Positive Cases:**                                                                        │
│                                                                                                                 │
│ 1. **Implications of distress or hopelessness**: Clients may not explicitly mention suicidal ideation but       │
│ express feelings of desperation, helplessness, or resignation.                                                  │
│ 2. **References to emotional pain or burden**: The text might contain hints at the client's emotional           │
│ struggles, such as feeling overwhelmed, trapped, or struggling with their condition.                            │
│                                                                                                                 │
│ **Concrete Suggestions to Modify the Prompt:**                                                                  │
│                                                                                                                 │
│ 1. **Incorporate phrases indicating distress or hopelessness**:                                                 │
│         * Add specific words or phrases that imply a sense of desperation or helplessness (e.g., "feeling       │
│ trapped," "overwhelmed," "can't cope").                                                                         │
│ 2. **Broaden language associated with suicidal ideation**:                                                      │
│         * Include more general terms related to emotional pain, such as "emotional burden," "weight on their    │
│ shoulders," or "too much to bear."                                                                              │
│ 3. **Emphasize contextual understanding**:                                                                      │
│         * Clarify that the prompt is not solely focused on explicit suicidality language but also considers the │
│ client's overall situation and emotional state.                                                                 │
│ 4. **Consider past behavior and history**:                                                                      │
│         * Weigh previous suicidal ideation or attempts against current circumstances, rather than relying       │
│ solely on current text.                                                                                         │
│                                                       

╭──────────────────────────────────────────── True Positives Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│ **Analysis of True Positive Examples**                                                                          │
│                                                                                                                 │
│ ### 1. Common Patterns or Characteristics in These True Positives                                               │
│                                                                                                                 │
│ Upon analyzing the true positive examples, several common patterns emerge:                                      │
│                                                                                                                 │
│ * **Explicit mention of self-harm or suicidal thoughts**: All three examples explicitly mention the consumer's  │
│ struggles with harming themselves (Example 1), a previous suicide attempt (Example 2), or feeling hopeless and  │
│ struggling to cope (Example 3).                                                                                 │
│ * **Expression of emotional distress**: The consumers in all three examples express feelings of hopelessness,   │
│ difficulty coping, or a sense of impending doom.                                                                │
│ * **Direct language related to self-harm or suicidal ideation**: The language used is direct and unambiguous,   │
│ indicating a clear risk of self-harm or suicidal behavior.                                                      │
│                                                                                                                 │
│ ### 2. Key Elements of the Current Prompt that Effectively Capture These Positive Cases                         │
│                                                                                                                 │
│ The prompt effectively captures these positive cases by:                                                        │
│                                                                                                                 │
│ * **Specifically asking about suicidal thoughts or ideation**: The prompt mentions "suicidal thoughts" and      │
│ "ideation," which directly correlates with the language used in Examples 1-3.                                   │
│ * **Inquiring about self-harm or physical damage**: The prompt asks if the consumer implies they might do       │
│ physical damage to themselves or property, which is relevant to Example 2 (previous suicide attempt).           │
│ * **Looking for explicit statements**: The prompt encourages a literal interpretation of the text, focusing on  │
│ explicit statements that indicate the presence or absence of the risk.                                          │
│                                                                                                                 │
│ ### 3. Suggestions on How to Reinforce These Successful Patterns in the Prompt                                  │
│                                                                                                                 │
│ To reinforce these successful patterns, consider:                                                               │
│                                                                                                                 │
│ * **Emphasizing the importance of explicit language**: Reiterate that the prompt is looking for direct and      │
│ unambiguous language related to self-harm or suicidal ideation.                                                 │
│ * **Specifying emotional distress indicators**: Add language to the prompt that highlights the importance of    │
│ detecting consumers who express feelings of hopelessne

╭─────────────────────────────────────────── Invalid Outputs Analysis ────────────────────────────────────────────╮
│                                                                                                                 │
│ **Analysis of Invalid Outputs**                                                                                 │
│                                                                                                                 │
│ After reviewing the provided invalid outputs, I've identified some common patterns and characteristics:         │
│                                                                                                                 │
│ 1. **Lack of relevance**: The first two outputs (conversation starters) are not relevant to assessing risk      │
│ factors, which is the primary objective of the prompt.                                                          │
│ 2. **Unclear context**: In the third output, there's an attempt to shift the conversation focus from a customer │
│ service issue to assessing risk factors, but it's unclear and lacks context.                                    │
│ 3. **Insufficient information**: The fourth output (empty JSON) indicates that there's no relevant information  │
│ about suicidal thoughts or ideation, self-harm, or other risk factors.                                          │
│                                                                                                                 │
│ **Specific Elements Contributing to Formatting Errors**                                                         │
│                                                                                                                 │
│ The following elements of the current prompt and output format instructions might be causing these formatting   │
│ errors:                                                                                                         │
│                                                                                                                 │
│ 1. **Ambiguity in prompt**: The initial question "Please look for the following risk factor" could be           │
│ interpreted as a general inquiry rather than a specific assessment task.                                        │
│ 2. **Lack of clear context**: There's no explicit mention of the context or scenario in which the risk          │
│ assessment should take place (e.g., clinical, social work, or other professional settings).                     │
│ 3. **Output format instructions**: The JSON output format is strict and doesn't account for scenarios where     │
│ there might not be sufficient information to provide a meaningful response.                                     │
│                                                                                                                 │
│ **Suggestions for Improving Prompt and Output Format Instructions**                                             │
│                                                                                                                 │
│ To reduce invalid outputs, I recommend the following modifications:                                             │
│                                                                                                                 │
│ 1. **Clarify prompt**: Revise the prompt to explicitly state that it's a risk assessment task in a specific     │
│ context (e.g., "Assessing potential risks of self-harm in a clinical setting").                                 │
│ 2. **Specify scenario**: Add a brief description of the scenario or context in which the risk assessment should │
│ take place.                                                                                                     │
│ 3. **Provide clear guidelines**: Emphasize that the output format instructions are strict and require providing │
│ a meaningful response, even if there's no relevant inf